In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
Gemini_key = os.getenv("Gemini_key")
Typhoon_key = os.getenv("Typhoon_key")


In [17]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_core.documents import Document
import re
file_paths = [
    ("Data/ปฏิทินการศึกษา2568.pdf", "calendar"),
    ("Data/รายละเอียดของหลักสูตร.pdf", "curriculum"),
]
def clean_text(text):
    text = text.replace('\uf70b', '้') 
    text = text.replace('\uf70a', '่')
    text = text.replace('\uf701', '')
    text = text.replace('\uf712', '็')
    text = text.replace('\uf70e', '์')  
    text = text.replace('\uf710', 'ั')   
    text = text.replace('\uf702', 'ำ')  
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()
docs = []
for path, source_tag in file_paths:
    loader = PDFPlumberLoader(path)
    loaded_pages = loader.load()
    for i, page in enumerate(loaded_pages):
        raw_text = page.page_content
        cleaned = clean_text(raw_text)
        doc = Document(
            page_content=cleaned,
            metadata={
                "page": i + 1,
                "source": source_tag,
                "file": os.path.basename(path),
            }
        )
        docs.append(doc)

In [14]:
print(f"Loaded {len(docs)} documents from {len(file_paths)} files.")

Loaded 164 documents from 2 files.


In [18]:
docs[105]

Document(metadata={'page': 104, 'source': 'curriculum', 'file': 'รายละเอียดของหลักสูตร.pdf'}, page_content='2. การประเมินทักษะของอาจารย์ในการใช้แผนกลยุทธ์การสอน - ประเมินจากนักศึกษาเกี่ยวกับการสอนของอาจารย์ในทุกด้าน เช่น กลวิธีการสอน การตรง ต่อเวลา การชี้แจงเป\uf706าหมาย วัตถุประสงค์ของรายวิชา เกณฑ์การวัดและประเมินผล และ การใช้สื่อการสอน - ประเมินโดยตัวอาจารย์เองและเพื่อนร่วมงาน 6.4.3.2 การประเมินหลักสูตรในภาพรวม มีกระบวนการที่ได้ข้อมูลย้อนกลับในการประเมินคุณภาพของหลักสูตรในภาพรวม เช่น 1. ประเมินหลักสูตรในภาพรวมโดยการทำแบบสอบถามนักศึกษาชั้นปำสุดท้ายและบัณฑิตใหม่ เกี่ยวกับสถานภาพการประกอบอาชีพ 2. ประเมินโดยที่ปรึกษาหรือผู้ทรงคุณวุฒิจากรายงานผลการดำเนินการหลักสูตรประจำปำ 3. ประเมินโดยผู้ใช้บัณฑิตหรือผู้มีส่วนเกี่ยวข้องอื่นๆ 4. สำรวจจำนวนนักศึกษาที่สำเร็จการศึกษา สำรวจข้อมูลเกี่ยวกับการทำงานและการศึกษาต่อ ของบัณฑิต 6.4.3.3 การประเมินผลการดำเนินงานตามรายละเอียดหลักสูตร เป็นไปตามการประเมินคุณภาพหลักสูตรตามหลักเกณฑ์ของ สกอ. 6.4.3.4 การทบทวนผลการประเมินและวางแผนปรับปรุง 1. มีการนำข้อมูลจากการ

In [19]:
# แบ่งเอกสารเป็นชิ้น
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
    is_separator_regex=False,
)
splits = text_splitter.split_documents(docs)

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=Gemini_key
)
vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embedding
)

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=Gemini_key
)

In [38]:

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.tools.retriever import create_retriever_tool

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_kwargs={"k": 6}),
    llm=llm
)

retriever_tool = create_retriever_tool(
    retriever=multi_query_retriever, # ใช้ retriever ตัวใหม่
    name="university_document_search",
    description="ใช้สำหรับค้นหาข้อมูลจากเอกสารเกี่ยวกับรายละเอียดของหลักสูตรวิทยาการคอมพิวเตอร์และปฏิทินการศึกษา"
)

In [40]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# สร้าง Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "คุณคือผู้ช่วย AI ที่เชี่ยวชาญด้านข้อมูลหลักสูตรและปฏิทินการศึกษาของมหาวิทยาลัย\n"
     "ผู้ใช้ทุกคนถามเกี่ยวกับหลักสูตรหรือเอกสารที่คุณมี\n"
     "คุณต้องตอบคำถามโดยใช้ข้อมูลจากเครื่องมือ 'university_document_search' เท่านั้น ห้ามใช้ความรู้ภายนอกเด็ดขาด\n"
     "1.วิเคราะห์คำถาม: ทำความเข้าใจว่าผู้ใช้ต้องการทราบข้อมูลอะไร เช่น รหัสวิชา, หน่วยกิต, วันที่, หรือข้อมูลภาพรวม\n"
     "2.ใช้เครื่องมือ: เรียกใช้ 'university_document_search' เพื่อค้นหาข้อมูลที่เกี่ยวข้องที่สุด\n"
     "3.สังเคราะห์คำตอบ: อ่านข้อมูลที่เครื่องมือค้นหามาให้ทั้งหมดอย่างละเอียด แม้ข้อมูลจะดูไม่เป็นระเบียบหรือมาจากตารางที่แตกไปบ้าง ให้พยายามทำความเข้าใจและสกัดเฉพาะส่วนที่ตอบคำถามของผู้ใช้\n"
     "4.ตอบอย่างมั่นใจ: ตอบเป็นภาษาไทยที่กระชับและชัดเจน หากไม่พบข้อมูลจริงๆ ให้ตอบว่า 'ไม่พบข้อมูลที่เกี่ยวข้องในเอกสาร' ห้ามเดาคำตอบ จัดรูปคำตอบให้กระชับและตรงประเด็น\n"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
# สร้าง Agent
tools = [retriever_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [34]:
def verify_answer(llm, question: str, answer: str, context: str) -> bool:
    verify_prompt = f"""
    คุณเป็นผู้ตรวจสอบความถูกต้องของคำตอบจากเอกสารมหาวิทยาลัย
    คำถาม: {question}
    คำตอบ: {answer}
    ข้อมูลอ้างอิงจากเอกสาร: {context}
    กรุณาเช็คอย่างละเอียด คำตอบข้างต้น มีข้อมูลสนับสนุนอยู่ในข้อความอ้างอิงหรือไม่:
    - ถ้ามีข้อมูลสนับสนุนให้ตอบว่า "YES"
    - ถ้าบางส่วนของคำตอบไม่มีข้อมูลสนับสนุนในข้อความอ้างอิง หรือคำตอบ **ไม่ถูกต้องตามข้อมูลอ้างอิง** ให้ตอบว่า "NO"
    - หากคำตอบเป็นการระบุว่า 'ไม่พบข้อมูล' ให้พิจารณาว่า **ถูกต้อง** ถ้าบริบทไม่มีข้อมูลนั้นจริงๆ ให้ตอบว่า "YES"
    ตอบเฉพาะ "YES" หรือ "NO" เท่านั้น
    """
    try:
        response = llm.invoke(verify_prompt)
        result = response.content.strip().upper()
        return result.startswith("YES")
    except Exception as e:
        print("ตรวจสอบคำตอบล้มเหลว:", e)
        return False



In [41]:
question = "มีกลุ่มวิชาโทอะไรบ้าง"
response = agent_executor.invoke({"input": question})
raw_answer = response["output"]

retrieved_docs = vectordb.similarity_search(question, k=6)
if not retrieved_docs:
    print("ไม่พบข้อมูลที่เกี่ยวข้องในเอกสาร")
else:
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    #print(f"คำตอบจาก Agent: {raw_answer}")
    #print(f"Context ที่ใช้ตรวจสอบ: \n{context}...") 
    is_valid = verify_answer(llm, question, raw_answer, context)
    if is_valid:
        print(raw_answer)
    else:
        print("ไม่สามารถตอบได้")



> Entering new AgentExecutor chain...

Invoking: `university_document_search` with `{'query': 'กลุ่มวิชาโท'}`


มคอ.2 6.5 อาจารย์ผู้รับผิดชอบหลักสูตรและอาจารย์ประจำหลักสูตร วิชาเอก คอมพิวเตอร์และวิทยาการสารสนเทศ (ภาคปกติ) ลำดับ ตำแหน่งทาง สำเร็จการศึกษาจาก ชื่อ - สกุล คุณวุฒิ สาขาวิชา ที่ วิชาการ สถาบัน ปำ พ.ศ. 1. ผู้ช่วย ดร.วนิดา พฤทธิวิทยา Ph.D. Computer Science Iowa State University, USA. 2549 ศาสตราจารย์ M.S. Computer Science University of Southern California, 2543 USA. วท.บ. ศาสตร์คอมพิวเตอร์ มหาวิทยาลัยธรรมศาสตร์ 2539 (เกียรตินิยมอันดับหนึ่ง เหรียญทอง) 2. อาจารย์ นุชชากร งามเสาวรส พบ.ม. สถิติประยุกต์ สถาบันบัณฑิตพัฒนบริหารศาสตร์ 2534 วท.บ. คณิตศาสตร์ มหาวิทยาลัยสงขลานครินทร์ 2530 (เกียรตินิยมอันดับสอง) 3. อาจารย์ สิริกันยา นิลพานิช M.Sc. Computer Science Syracuse University, USA. 2538 วท.บ. ศาสตร์คอมพิวเตอร์ มหาวิทยาลัยธรรมศาสตร์ 2533 (เกียรตินิยมอันดับสอง) 105

ศศิธร เกรียงไกรวณิช, สุปราณี เทศขวัญ และ ธนาธร ทะนานทอง (2019), “ระบบติดตามและวิเคราะห์ ความคิดเห็นต่อมหาวิทยาลัยไทยบ